# Collaborative Filtering from Scratch

The goal is to implement a collaborative filtering algorithm from scratch. We should have the following progression:\
- User-Item Collaborative Filtering
    - Basics
    - CF algorithm
    - Naive collaborative filtering with Alternating Least Squares
    - Naive CF with SGD
    - Updating a CF recommendation system
- Item-Item Collaborative Filtering
    - Basics
    - Algorithm
    - Implementation
    - Notes
- NN Collaborative Filtering
    - Basics
    - From Matrix Operations to NN
    - Naive NNCF
    - additional NNCF flavors

#### Extras
But modern CF models use neural networks...

- NN for CF with side data
- Bayesian CF

#### A good flavor of CF: Tag-based CF

If we have time, we can make a more efficient recommendation system that's
- more storage efficient
- good for online training scenario

### Basics

The recommender problem is one of **matrix completion**

We start with a rating matrix $R$, which is $N$ x $M$ ($n$ users, $m$ items)

Let's say our target is the predicted ratings  $\hat{Y}_{ui} \approx R$. Then our optimization problem becomes
$$ \mathcal{L}(Z) = \sum_{ij:Y_{ij \neq ?}} ||Z - Y||^2$$

With some constraints, we can make the problem solvable:
- $Y$ is low rank $\rightarrow Z = UV^T \approx Y$
- We can map $U$ and $V$ pair-wise to latent variables $\rightarrow U$ is [$N$ x $K$], $V$ is [$M$ x $K$]

This also means we need to "fill out" $U$ and $V$.

For basic collaborative filtering with matrices, we can do this with Alternating Least Squares (ALS).

Formally, we have to solve $$argmin_{U, V}  (R - UV^T + C)$$
where $C$ abbreviates the regularization.

#### Simple ALS

We can solve the normal equations
$$ U = (V V^T + \lambda I)^{-1} V R$$
$$ V = (U U^T + \lambda I)^{-1} U R$$

to minimize the RMSE for our reconstructed $R$.

In [5]:
import numpy as np

In [6]:
#2 users, 3 items, 5 latents
n = 2
m = 3
k = 5
noise = 1e-4
#Generates [2,3] array
R = np.array([[0, 1, 0], [1, 1, 0]], dtype=float)

In [7]:
#Alternating Least Squares
best_U = np.random.randn(n,k)
best_Vt = np.random.randn(m,k)
#We can do this in closed form

#Solve for V first, by fixing U
best_Vt = np.linalg.inv(best_U.T @ best_U + noise*np.eye(k)) @ best_U.T @ R
#Solve for U, by fixing V
best_U = (np.linalg.inv(best_Vt @ best_Vt.T + noise*np.eye(k)) @ best_Vt @ R.T).T

In [8]:
#Check RMSE
np.sqrt(np.mean((R - (best_U @ best_Vt))**2))

0.00025311028536202575

This is linear ALS, we can improve our results with non-linear methods like PMF (Probabilistic Matrix Factorization) later.

#### ALS using Stochastic Gradient Descent

We can use Gradient Descent as well to generate our new $R$

Let our loss function be 
$$\mathcal{L}(R) = (R - U V^T)^2$$
and $$ R = UV^T $$

We solve for these so that we can apply the chain rule:
$$ \frac{\partial\mathcal{L}}{\partial R} = 2 (R - UV^T)$$
$$ \frac{\partial R}{\partial U} = V$$
$$ \frac{\partial R}{\partial V} = U$$

Then we get our gradients:
$$ \frac{\partial\mathcal{L}}{\partial U} = \frac{\partial\mathcal{L}}{\partial R} \frac{\partial\mathcal{R}}{\partial U}$$


$$ \frac{\partial\mathcal{L}}{\partial V} = \frac{\partial\mathcal{L}}{\partial R} \frac{\partial\mathcal{R}}{\partial V}$$

For every parameter $\theta$ and learning rate $\alpha$, we update during each iteration $i$ (gradient descent):
$$ \theta_{i+1} = \theta_i - \alpha \frac{\partial\mathcal{L}}{\partial \theta}$$

In [80]:
#TODO: Code here.
lr = 0.05
n_iter = 100
best_U = np.random.randn(n,k)
best_V = np.random.randn(m,k)
print(best_U @ best_V.T)
#Gradient Descent
for _iter in range(n_iter):
    #Solve for U
    best_U  = best_U + lr*(2*(R - best_U @ best_V.T)@ best_V)
    #Solve for V
    best_V = best_V + lr*(2*(R - best_U @ best_V.T).T @ best_U)
    if(np.linalg.norm(R - (best_U @ best_V.T)) < 1e-2):
        print("converged in " + str(_iter) + " iterations")
        break
print(best_U @ best_V.T)
R

[[-4.45893958 -2.14089102  3.98616959]
 [ 6.86761432  1.71099814  2.86743179]]
converged in 26 iterations
[[ 3.06696587e-03  9.92514933e-01  3.92092960e-04]
 [ 9.99778792e-01  1.00053987e+00 -2.82800717e-05]]


array([[0., 1., 0.],
       [1., 1., 0.]])

For the streaming case, we can selectively update each entry of $R_{ui}$ iteratively with SGD.

## Item-Item Collaborative Filtering

In practice, item-item CF works better. Items are simpler than human beings (and change less often)

Inputs:
- User-Item Utility Matrix
- Item Similarity Mappings

Outputs:
- Recommendation


Steps:
- Find arbitrary number of similar items to ones the user liked
- Rank them and serve the top 3 (that user hasn't seen yet)


#### Complexity
Assume that we already have the utility matrix. Given $m$ items, we would have $O(m)$ complexity for 1 query from a user.

In [ ]:
#TODO: Code here


#Data initialization



#Item Similarity Mappings



#
##Algorithm...
#

#Find similar items to each item user enjoyed


#Rank and serve

## NN Collaborative Filtering

We can also use a neural network to implement a recommender system. TBD

In [ ]:
#TODO: Code here